# Day 7 - Lab 2: Agent Interoperability with A2A Protocol

**Objective:** Master agent-to-agent communication using the A2A protocol for building collaborative multi-agent onboarding systems.

**Estimated Time:** 135 minutes

## Introduction

In this lab, you'll implement the A2A (Agent-to-Agent) protocol from scratch using pure Python. You'll build specialized agents that collaborate to onboard new employees.

### Why A2A?

Modern AI systems require multiple specialized agents:
- **DocumentationAgent**: Answers policy questions
- **AccessSetupAgent**: Manages system permissions
- **TrainingScheduleAgent**: Plans learning paths
- **CoordinatorAgent**: Orchestrates the workflow

A2A enables these agents to:
1. **Discover** each other's capabilities
2. **Communicate** with structured messages
3. **Collaborate** on complex tasks

By the end, you'll have a working multi-agent onboarding system.

## Step 1: Environment Setup (No External Dependencies!)

In [ ]:
import sys
import os
import json
import uuid
import time
import asyncio
from datetime import datetime, timedelta
from dataclasses import dataclass, field, asdict
from typing import List, Dict, Optional, Any, Callable
from enum import Enum
from collections import defaultdict
import queue
import threading

# Add project root to path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

from utils import setup_llm_client, save_artifact
client, model_name, api_provider = setup_llm_client(model_name='gpt-4o')

print('✅ Environment ready - Pure Python A2A implementation')

## Step 2: Load Onboarding Data

In [ ]:
# Load shared onboarding data
assets_path = 'assets'

with open(f'{assets_path}/onboarding_docs.json', 'r') as f:
    onboarding_docs = json.load(f)

with open(f'{assets_path}/roles_access_matrix.json', 'r') as f:
    roles_access = json.load(f)

with open(f'{assets_path}/training_catalog.json', 'r') as f:
    training_catalog = json.load(f)

with open(f'{assets_path}/new_hires_sample.json', 'r') as f:
    new_hires = json.load(f)

print(f'Loaded data for {len(new_hires["new_hires"])} new hires')

## Step 3: Define A2A Protocol Components

We'll build the protocol from scratch:

In [ ]:
# A2A Message Types
class MessageType(Enum):
    ANNOUNCE = 'announce'
    REQUEST = 'request'
    RESPONSE = 'response'
    ERROR = 'error'
    EVENT = 'event'

# A2A Message Envelope
@dataclass
class A2AMessage:
    protocol: str = 'a2a.v1'
    msg_id: str = field(default_factory=lambda: str(uuid.uuid4()))
    corr_id: Optional[str] = None
    type: MessageType = MessageType.REQUEST
    sender: str = ''
    recipient: str = ''
    capabilities: List[str] = field(default_factory=list)
    payload: Dict[str, Any] = field(default_factory=dict)
    ttl_ms: int = 10000
    timestamp: str = field(default_factory=lambda: datetime.now().isoformat())
    
    def to_dict(self) -> Dict:
        d = asdict(self)
        d['type'] = self.type.value
        return d

print('✅ A2A Protocol components defined')